# 106 LangGraph: Conditional Routing - Decision-Making Graphs

**Workshop**: LangGraph 101
**Duration**: ~20 minutes
**Difficulty**: Intermediate

## Learning Objectives

By completing this notebook, you will:
- Understand the `add_conditional_edges()` API and how it enables dynamic routing
- Create router functions with `Literal` return types for type-safe routing
- Implement branching workflows based on state conditions
- Master the lambda passthrough pattern (`lambda state: state`) for router nodes
- Route SCM workflows based on address types (IP vs FQDN vs Range)
- Apply network ACL match logic analogy to understand conditional routing
- Build decision trees similar to firewall rule processing

## Prerequisites

- **Knowledge**: Completed Notebooks 103 (State & Reducers), 104 (Sequential Graphs), and 105 (Graph Visualization)
- **Concepts**: Understanding of TypedDict, StateGraph, nodes, and edges
- **Setup**: None required for this notebook

## Table of Contents

1. [Introduction](#1-introduction)
2. [Journey So Far](#2-journey-so-far)
3. [Network ACL Analogy](#3-network-acl-analogy)
4. [Understanding Conditional Routing](#4-understanding-conditional-routing)
5. [Building Your First Conditional Graph](#5-building-your-first-conditional-graph)
6. [The Lambda Passthrough Pattern](#6-the-lambda-passthrough-pattern)
7. [SCM Address Type Routing](#7-scm-address-type-routing)
8. [Exercise: Build Your Own Router](#8-exercise-build-your-own-router)
9. [What's Next](#9-whats-next)
10. [Summary](#10-summary)

---

## 1. Introduction

Welcome to Notebook 106! In previous notebooks, you learned to build sequential workflows where state flows linearly from one node to the next. But real-world network automation often requires **decision-making** - routing workflows down different paths based on conditions.

### What is Conditional Routing?

**Conditional routing** allows your LangGraph workflows to make intelligent decisions about which path to take based on the current state. Instead of following a fixed sequence, your graph can branch into different paths depending on conditions you define.

Think about common SCM automation scenarios:
- **IF** folder is "Production" **THEN** require approval workflow **ELSE** create directly
- **IF** address type is "ip_netmask" **THEN** validate CIDR format **ELSE IF** type is "fqdn" **THEN** validate DNS format
- **IF** firewall has HA enabled **THEN** check HA sync status **ELSE** skip HA checks

This is exactly what conditional routing enables!

### Why This Matters

Without conditional routing:
- All workflows follow the same path regardless of conditions
- No way to handle different scenarios (dev vs prod, HA vs standalone)
- Logic becomes complex with nested if/else statements in nodes

With conditional routing:
- Workflows adapt intelligently based on state
- Clear visual representation of decision points
- Clean separation between routing logic and processing logic
- Easy to add new paths or modify routing conditions

Let's dive in!

---

## 2. Journey So Far

Let's recap what you've learned in the previous notebooks to understand where conditional routing fits:

### Notebook 103: State & Reducers

You learned that **state** is the foundation of LangGraph applications:
- Defined using `TypedDict` for type safety
- Flows through the entire workflow, accessible to all nodes
- Updated by nodes using **reducers** to merge changes
- Can track complex data like lists of SCM objects, validation errors, and API responses

**Key Concept**: State is your application's memory - it persists throughout the workflow.

### Notebook 104: Sequential Graphs

You learned to build **linear workflows** with nodes and edges:
- Created nodes as processing functions that update state
- Connected nodes with simple edges (`add_edge`) to define sequence
- Used `START` and `END` markers for entry and exit points
- Built workflows like: `START → validate → prepare → create → verify → END`

**Key Concept**: Edges define the flow from one node to the next.

### Notebook 105: Graph Visualization

You learned to visualize and understand your workflows:
- Generated Mermaid diagrams to see graph structure
- Identified nodes (processing units) and edges (connections)
- Debugged workflows by examining visual representations
- Understood how state flows through the graph

**Key Concept**: Visual graphs make complex workflows easy to understand.

### What's Missing? Decision-Making!

All the workflows you've built so far follow a **fixed path** - the same sequence of nodes executes every time. But what if you need to:
- Route to different validation logic based on address type?
- Skip approval for dev folders but require it for production?
- Handle HA firewalls differently than standalone firewalls?

That's where **conditional routing** comes in. It adds the final piece: **intelligent decision-making** based on state conditions.

Let's learn how to make our graphs smart!

---

## 3. Network ACL Analogy - Understanding Conditional Routing

### The ACL/Firewall Rule Processing Model

If you've worked with network firewalls or ACLs, you already understand conditional routing! Let's draw a direct parallel:

#### Traditional Firewall Rule Processing

When a packet arrives at a Palo Alto Networks firewall, it goes through a decision tree:

```
Packet arrives
    |
    v
Check Rule 1: source=10.0.0.0/8, dest=any, app=web-browsing
    |
    +--- MATCH? → Action: ALLOW → Log and forward packet
    |
    +--- NO MATCH? → Continue to Rule 2
            |
            v
Check Rule 2: source=192.168.0.0/16, dest=any, app=ssh
            |
            +--- MATCH? → Action: DENY → Drop packet
            |
            +--- NO MATCH? → Continue to Rule 3
                    |
                    v
                Default: DENY → Drop packet
```

### The LangGraph Equivalent

Conditional routing in LangGraph works exactly the same way, but instead of matching packet attributes, you're matching **state attributes**:

```
State arrives at router node
    |
    v
Check condition: state["folder"] == "Production"?
    |
    +--- TRUE? → Route to: approval_workflow node
    |
    +--- FALSE? → Check next condition
            |
            v
Check condition: state["folder"] == "Development"?
            |
            +--- TRUE? → Route to: direct_creation node
            |
            +--- FALSE? → Route to: error_handler node
```

### Key Parallels

| Network ACL Concept | LangGraph Concept | Example |
|---------------------|-------------------|----------|
| **Packet** | **State** | Data flowing through the system |
| **ACL Rule** | **Router Function** | Decision logic examining attributes |
| **Match Condition** | **if/elif/else** | Checking state fields |
| **Action (ALLOW/DENY)** | **Edge Name** | Which path to route to |
| **Next Hop** | **Next Node** | Where to send the data next |
| **Rule Processing** | **Conditional Edge** | The decision-making mechanism |

### Real-World Example: SCM Address Type Routing

Just as an ACL matches on packet attributes (source IP, dest port, protocol), your LangGraph router can match on state attributes:

**Firewall ACL Logic:**
```
IF source_ip in 10.0.0.0/8 THEN permit
ELSE IF source_ip in 192.168.0.0/16 THEN deny
ELSE default_deny
```

**LangGraph Router Logic:**
```python
def route_by_address_type(state):
    if state["address_type"] == "ip_netmask":
        return "validate_cidr"
    elif state["address_type"] == "fqdn":
        return "validate_dns"
    else:
        return "validate_range"
```

### Routing Tables Analogy

Another way to think about conditional routing is like a **routing table**:

```
Destination Network    | Next Hop      | Interface
----------------------|---------------|----------
10.0.0.0/8           | 192.168.1.1   | eth0
172.16.0.0/12        | 192.168.1.2   | eth1
0.0.0.0/0            | 192.168.1.254 | eth0 (default)
```

In LangGraph:

```
State Condition              | Next Node          | Edge Name
---------------------------|--------------------|--------------
requires_approval == True   | approval_workflow  | "approval_path"
requires_approval == False  | direct_creation    | "direct_path"
```

### Why This Analogy Matters

As network administrators, you're already experts at conditional logic:
- **ACLs** = Match conditions and take actions
- **Routing tables** = Destination-based forwarding decisions
- **Security policies** = Rule-based processing with first-match wins

**LangGraph conditional routing uses the exact same mental model** - you're just routing state through your workflow instead of routing packets through your network!

Now let's see this in action with code.

---

## 4. Understanding Conditional Routing

Now that we understand the ACL analogy, let's learn the LangGraph API for conditional routing.

### The Three Components

Conditional routing requires three components working together:

1. **Router Function**: Examines state and returns an edge name
2. **Lambda Passthrough Node**: Router node that doesn't modify state
3. **Conditional Edge**: Uses `add_conditional_edges()` to connect router to destinations

Let's explore each one!

In [ ]:
# Core imports
from typing import TypedDict, Literal
from pprint import pprint

# LangGraph imports
from langgraph.graph import StateGraph, START, END

# Visualization
from IPython.display import Image, display

print("✅ Imports successful!")
print("\nKey import: Literal from typing")
print("  - Used for type-safe router function return values")
print("  - Ensures router only returns valid edge names")

---

## 5. Building Your First Conditional Graph

Let's build a conditional graph step-by-step. We'll create an SCM folder-based routing workflow:
- **Production folders** require approval before creating objects
- **Development folders** allow direct creation

### Step 1: Define State

First, define state with fields needed for routing decisions:

### Production Pattern: ScmClient Initialization

Before building conditional routing workflows with real SCM API calls, you need to initialize the `ScmClient` once at the module level.

**Best Practice Pattern:**

In production LangGraph applications, initialize the SCM client once and reuse it across all nodes (including routing-based nodes). This avoids creating multiple client instances and maintains connection efficiency.

Here's the production pattern you'll use in real conditional workflows:

In [ ]:
# Production Pattern: Initialize ScmClient once at module level
# (This code is for demonstration - not executed in this notebook)

"""
from scm.client import Scm
import os

# Initialize once at the top of your module
scm_client = Scm(
    client_id=os.getenv("SCM_CLIENT_ID"),
    client_secret=os.getenv("SCM_CLIENT_SECRET"),
    tsg_id=os.getenv("SCM_TSG_ID")
)

# Then use in your routed processing nodes
def create_with_approval_production(state: ConditionalFolderState) -> dict:
    '''Production version using real ScmClient.'''
    
    # Use the module-level client (no need to create new instance)
    address_response = scm_client.address.create({
        "name": state["address_name"],
        "ip_netmask": "10.1.1.0/24",  # From state
        "folder": state["folder"]
    })
    
    return {
        "creation_path": "approved",
        "result": f"Created address with ID: {address_response['id']}"
    }


def create_without_approval_production(state: ConditionalFolderState) -> dict:
    '''Production version using real ScmClient.'''
    
    # Reuse the same client instance
    address_response = scm_client.address.create({
        "name": state["address_name"],
        "ip_netmask": "10.1.1.0/24",  # From state
        "folder": state["folder"]
    })
    
    return {
        "creation_path": "direct",
        "result": f"Created address with ID: {address_response['id']}"
    }

# Router functions work the same - they don't need the client
# They only make routing decisions, not API calls
def decide_creation_path_production(state: ConditionalFolderState) -> Literal["approval_path", "direct_path"]:
    '''Router function - no API calls, just routing logic.'''
    if state["requires_approval"]:
        return "approval_path"
    else:
        return "direct_path"
"""

print("💡 Production Pattern Explained:")
print("   1. Initialize ScmClient ONCE at module level")
print("   2. Store credentials in environment variables")
print("   3. Reuse the same client instance across all processing nodes")
print("   4. Router functions don't use the client - they only make decisions")
print("   5. Each routed processing node makes API calls using the shared client")
print("\n📚 For learning purposes, we'll use simulated versions below")
print("   Real API integration will be covered in later notebooks")

In [ ]:
class ConditionalFolderState(TypedDict):
    """State for conditional SCM folder-based routing."""
    address_name: str       # Address object name
    folder: str             # Target SCM folder
    requires_approval: bool # Does this folder require approval?
    creation_path: str      # Which creation path to follow
    result: str             # Outcome of the creation workflow

print("✅ ConditionalFolderState defined!")
print("\nState structure:")
print("  - address_name: str")
print("  - folder: str")
print("  - requires_approval: bool")
print("  - creation_path: str")
print("  - result: str")
print("\n💡 The 'requires_approval' field will be used for routing decisions!")

### Step 2: Create Processing Nodes

Now let's create TWO different processing nodes - one for the approval workflow path, one for direct creation:

**Key Concept**: These nodes perform different operations based on which path was taken.

In [ ]:
def create_with_approval(state: ConditionalFolderState) -> dict:
    """Node: Create address object with approval workflow (for production folders).

    Args:
        state: Current workflow state

    Returns:
        dict: Partial state update with creation result
    """
    address_name = state["address_name"]
    folder = state["folder"]

    # Simulate approval workflow
    result = f"Address '{address_name}' created in '{folder}' folder WITH approval workflow (Production path)"

    return {"creation_path": "approved", "result": result}


def create_without_approval(state: ConditionalFolderState) -> dict:
    """Node: Create address object directly (for non-production folders).

    Args:
        state: Current workflow state

    Returns:
        dict: Partial state update with creation result
    """
    address_name = state["address_name"]
    folder = state["folder"]

    # Simulate direct creation
    result = f"Address '{address_name}' created in '{folder}' folder DIRECTLY (Standard path)"

    return {"creation_path": "direct", "result": result}

print("✅ Both creation path functions defined!")
print("\n💡 Two different nodes - one for approved path, one for direct path")

### Step 3: Create the Router Function - THE DECISION POINT!

This is the most important part! The **router function** is where the decision-making happens.

#### Key Characteristics of Router Functions:

1. **Examines State**: Looks at state fields to make decisions
2. **Returns Edge Name**: Returns a string (not a dict!)
3. **Uses Literal Type**: Type hint specifies valid edge names
4. **No State Updates**: Doesn't modify state, only decides routing

#### The Literal Type Pattern - WHY IT MATTERS!

The `Literal` type provides **type safety** for your routing decisions. Let's see why this is critical:

**❌ WITHOUT Literal - No Type Safety:**
```python
def decide_path(state: MyState) -> str:
    if state["condition"]:
        return "approvel_path"  # TYPO! No IDE warning!
    return "direct_path"
```

**✅ WITH Literal - Type Safety Enabled:**
```python
def decide_path(state: MyState) -> Literal["approval_path", "direct_path"]:
    if state["condition"]:
        return "approvel_path"  # IDE ERROR: Not in Literal options!
    return "direct_path"
```

**Benefits of Literal:**
- 🔍 **IDE Autocomplete**: Your IDE suggests valid edge names
- 🚨 **Typo Detection**: Catches spelling mistakes at development time
- 📚 **Documentation**: Clearly shows valid routing paths
- 🛡️ **Type Safety**: Prevents runtime errors from invalid edge names

Think of `Literal` like an **ACL entry validation** - it ensures you only use valid actions (permit/deny), just like Literal ensures you only return valid edge names!

Let's create our router function with proper Literal typing:

### Step 3: Create the Router Function - THE CRITICAL PATTERN!

Now comes the most important part: the **router function**. This function makes the decision about which path to take.

⚠️ **CRITICAL DIFFERENCE**: Router functions are special! They DON'T modify state - they just return an **edge name** to indicate which path to follow.

**The Challenge**: LangGraph expects nodes to return state updates, but router functions return edge names. How do we handle this?

**The Solution**: The **lambda passthrough pattern** (explained in Section 6)!

In [ ]:
def decide_creation_path(state: ConditionalFolderState) -> Literal["approval_path", "direct_path"]:
    """Router function: Decide which creation path to take based on approval requirement.

    Args:
        state: Current workflow state

    Returns:
        Literal["approval_path", "direct_path"]: Edge name indicating which path to follow
        
    Note:
        The Literal return type provides:
        - IDE autocomplete for return values
        - Type checking to prevent typos
        - Clear documentation of valid paths
    """
    # Check if approval is required
    if state["requires_approval"]:
        return "approval_path"  # Route to approval workflow
    else:
        return "direct_path"    # Route to direct creation

print("✅ decide_creation_path router function defined!")
print("\n💡 Key differences from regular nodes:")
print("   1. Returns EDGE NAME (string), not state update (dict)")
print("   2. Uses Literal type hint for type safety")
print("   3. Doesn't modify state - only makes routing decision")
print("\n🔍 Literal['approval_path', 'direct_path'] ensures:")
print("   - IDE autocomplete suggests these exact strings")
print("   - Typos like 'approvel_path' are caught at development time")
print("   - Clear documentation of valid routing options")

In [ ]:
# Step 1: Create the graph
folder_routing_graph = StateGraph(ConditionalFolderState)

# Step 2: ⭐ CRITICAL - Add router node with lambda passthrough
# The lambda passthrough pattern: lambda state: state
# This satisfies LangGraph's requirement that nodes return state
# while letting the router function (used in path=) make routing decisions
folder_routing_graph.add_node("router", lambda state: state)

# Step 3: Add both processing nodes
folder_routing_graph.add_node("create_with_approval", create_with_approval)
folder_routing_graph.add_node("create_direct", create_without_approval)

# Step 4: Connect START to router
folder_routing_graph.set_entry_point("router")

# Step 5: 🌟 ADD CONDITIONAL EDGE - This is the key API!
# Three parameters:
#   1. source: Which node makes the decision? ("router")
#   2. path: Which function decides the route? (decide_creation_path)
#   3. path_map: Map edge names to destination nodes
folder_routing_graph.add_conditional_edges(
    source="router",                      # Router node decides
    path=decide_creation_path,            # Using this function
    path_map={                            # Map edge names → nodes
        "approval_path": "create_with_approval",
        "direct_path": "create_direct"
    }
)

# Step 6: Connect both processing nodes to END
folder_routing_graph.add_edge("create_with_approval", END)
folder_routing_graph.add_edge("create_direct", END)

# Step 7: Compile
folder_routing_app = folder_routing_graph.compile()

print("✅ Conditional folder routing graph built and compiled!")
print("\n💡 Flow: START → router → [approval OR direct path] → END")
print("\n🔑 Key API: add_conditional_edges(source, path, path_map)")

### Step 4: Visualize the Conditional Graph

Let's see our branching workflow:

In [ ]:
# Visualize the conditional routing graph
display(Image(folder_routing_app.get_graph().draw_mermaid_png()))

print("\n📊 Graph Structure Analysis:")
print("=" * 60)
print("Nodes:")
print("  - router (decision point)")
print("  - create_with_approval (processing)")
print("  - create_direct (processing)")
print("\nEdges:")
print("  - START → router")
print("  - router → create_with_approval (when requires_approval=True)")
print("  - router → create_direct (when requires_approval=False)")
print("  - Both processing nodes → END")
print("\nDecision Logic:")
print("  Field: state['requires_approval']")
print("  True path: Production folders requiring approval")
print("  False path: Development folders allowing direct creation")

### Step 5: Test with Approval Path (Production Folder)

Let's test the approval path:

In [ ]:
# Test the approval path (production folder)
result = folder_routing_app.invoke({
    "address_name": "prod_server",
    "folder": "Production",
    "requires_approval": True,
    "creation_path": "",
    "result": ""
})

print("✅ Approval path executed!")
print("\nResult:")
print(result["result"])
print(f"\nPath taken: {result['creation_path']}")

### Step 6: Test with Direct Path (Development Folder)

Now let's test the direct creation path:

---

## 6. The Lambda Passthrough Pattern

Let's dive deep into one of the most important patterns in LangGraph: **the lambda passthrough pattern**.

### The Problem

LangGraph has a fundamental requirement: **all nodes must return state updates** (as dictionaries). But router functions need to return **edge names** (as strings) to indicate routing decisions.

This creates a conflict:

```python
# Regular nodes return dicts
def regular_node(state: MyState) -> dict:
    return {"field": "updated value"}  # ✅ Returns dict

# Router functions return strings
def router_function(state: MyState) -> str:
    if state["condition"]:
        return "edge_a"  # ❌ Returns string, not dict!
    return "edge_b"
```

### The Solution: Lambda Passthrough

The pattern `lambda state: state` solves this elegantly:

```python
# ❌ WRONG - Router function doesn't return dict
graph.add_node("router", router_function)

# ✅ CORRECT - Lambda returns state unchanged
graph.add_node("router", lambda state: state)
```

### How It Works

The lambda passthrough separates two responsibilities:

1. **Node responsibility** (lambda state: state):
   - Satisfies LangGraph's requirement that nodes return state
   - Doesn't modify state - just passes it through unchanged
   - Think: "I'm a node that does nothing to state"

2. **Router responsibility** (path= parameter):
   - Makes the actual routing decision
   - Examines state and returns edge name
   - Think: "I decide which path to take"

### Visual Breakdown

```python
# Add the router node
graph.add_node("router", lambda state: state)  # ← Node: pass state through

# Add conditional edge
graph.add_conditional_edges(
    source="router",              # ← From the router node
    path=decide_creation_path,    # ← Router function: makes decision
    path_map={...}                # ← Maps edge names to nodes
)
```

### Network Analogy

Think of this like a router with a **control plane** and **data plane**:

- **Lambda (data plane)**: Forwards packets (state) without modification
- **Router function (control plane)**: Makes forwarding decisions (routing logic)
- **path_map (routing table)**: Maps destinations to next hops

### Key Takeaway

**Always use `lambda state: state` when adding router nodes!**

This is the standard pattern for all conditional routing in LangGraph. The lambda handles state (pass through), while the router function (in `path=`) handles routing decisions.

```python
# The complete pattern
graph.add_node("router", lambda state: state)  # Passthrough
graph.add_conditional_edges(
    source="router",
    path=your_router_function,  # Decision logic
    path_map={"edge": "node"}   # Routing table
)
```

---

## 7. SCM Address Type Routing

Let's apply what we've learned to a real SCM automation scenario: routing based on address object types.

### The Scenario

SCM address objects come in three types:
1. **ip_netmask**: IP address with CIDR notation (e.g., 10.1.1.0/24)
2. **fqdn**: Fully qualified domain name (e.g., example.com)
3. **ip_range**: IP address range (e.g., 10.1.1.1-10.1.1.254)

Each type requires different validation logic. We'll use conditional routing to send each type to its appropriate validator.

### The Routing Logic (ACL-style)

Think of this like an ACL:

```
IF address_type == "ip_netmask" THEN route_to → validate_cidr
ELSE IF address_type == "fqdn" THEN route_to → validate_fqdn  
ELSE IF address_type == "ip_range" THEN route_to → validate_range
```

Let's build it!

In [ ]:
# Define state for address type routing
class AddressTypeState(TypedDict):
    """State for routing SCM address objects by type."""
    name: str
    address_type: str  # "ip_netmask", "fqdn", or "ip_range"
    address_value: str
    folder: str
    validated: bool
    validation_result: str

print("✅ AddressTypeState defined")
print("\nThis state tracks address type and routes to appropriate validator")

In [ ]:
# Router function with Literal return type
def route_by_address_type(state: AddressTypeState) -> Literal["cidr_path", "fqdn_path", "range_path"]:
    """Route based on address type.
    
    Returns:
        Edge name indicating which validation path to take
    """
    addr_type = state["address_type"]
    
    if addr_type == "ip_netmask":
        print(f"📍 Routing decision: {addr_type} → cidr_path")
        return "cidr_path"
    elif addr_type == "fqdn":
        print(f"📍 Routing decision: {addr_type} → fqdn_path")
        return "fqdn_path"
    else:  # ip_range
        print(f"📍 Routing decision: {addr_type} → range_path")
        return "range_path"

# Validator nodes
def validate_cidr(state: AddressTypeState) -> dict:
    """Validate IP/netmask format."""
    import re
    value = state["address_value"]
    pattern = r'^(\d{1,3}\.){3}\d{1,3}/\d{1,2}$'
    
    is_valid = bool(re.match(pattern, value))
    result = f"CIDR validation: {value} is {'valid' if is_valid else 'invalid'}"
    
    print(f"✓ {result}")
    return {"validated": is_valid, "validation_result": result}

def validate_fqdn(state: AddressTypeState) -> dict:
    """Validate FQDN format."""
    import re
    value = state["address_value"]
    pattern = r'^([a-zA-Z0-9]([a-zA-Z0-9\-]{0,61}[a-zA-Z0-9])?\.)+[a-zA-Z]{2,}$'
    
    is_valid = bool(re.match(pattern, value))
    result = f"FQDN validation: {value} is {'valid' if is_valid else 'invalid'}"
    
    print(f"✓ {result}")
    return {"validated": is_valid, "validation_result": result}

def validate_range(state: AddressTypeState) -> dict:
    """Validate IP range format."""
    import re
    value = state["address_value"]
    pattern = r'^(\d{1,3}\.){3}\d{1,3}-(\d{1,3}\.){3}\d{1,3}$'
    
    is_valid = bool(re.match(pattern, value))
    result = f"Range validation: {value} is {'valid' if is_valid else 'invalid'}"
    
    print(f"✓ {result}")
    return {"validated": is_valid, "validation_result": result}

print("✅ Router and validator nodes defined")
print("\nRouting logic:")
print("  ip_netmask → validate_cidr")
print("  fqdn → validate_fqdn")
print("  ip_range → validate_range")

In [ ]:
# Build the address type routing graph
address_type_graph = StateGraph(AddressTypeState)

# Add router node with lambda passthrough
address_type_graph.add_node("router", lambda state: state)

# Add validator nodes
address_type_graph.add_node("validate_cidr", validate_cidr)
address_type_graph.add_node("validate_fqdn", validate_fqdn)
address_type_graph.add_node("validate_range", validate_range)

# Set entry point
address_type_graph.set_entry_point("router")

# Add conditional edges with 3-way routing
address_type_graph.add_conditional_edges(
    source="router",
    path=route_by_address_type,
    path_map={
        "cidr_path": "validate_cidr",
        "fqdn_path": "validate_fqdn",
        "range_path": "validate_range"
    }
)

# Connect validators to END
address_type_graph.add_edge("validate_cidr", END)
address_type_graph.add_edge("validate_fqdn", END)
address_type_graph.add_edge("validate_range", END)

# Compile
address_type_app = address_type_graph.compile()

print("✅ Address type routing graph compiled")
print("\n💡 This graph has 3 possible paths based on address_type!")

In [ ]:
# Visualize the 3-way routing
display(Image(address_type_app.get_graph().draw_mermaid_png()))

print("\n📊 Notice the 3-way branch from the router node!")
print("   This is like a routing table with 3 destination networks")

In [ ]:
# Test all three paths
print("=" * 60)
print("TEST 1: IP/Netmask Address")
print("=" * 60)

result1 = address_type_app.invoke({
    "name": "internal-network",
    "address_type": "ip_netmask",
    "address_value": "10.1.1.0/24",
    "folder": "Texas",
    "validated": False,
    "validation_result": ""
})

print(f"\nResult: {result1['validation_result']}")
print(f"Validated: {result1['validated']}")

print("\n" + "=" * 60)
print("TEST 2: FQDN Address")
print("=" * 60)

result2 = address_type_app.invoke({
    "name": "example-site",
    "address_type": "fqdn",
    "address_value": "example.com",
    "folder": "Texas",
    "validated": False,
    "validation_result": ""
})

print(f"\nResult: {result2['validation_result']}")
print(f"Validated: {result2['validated']}")

print("\n" + "=" * 60)
print("TEST 3: IP Range Address")
print("=" * 60)

result3 = address_type_app.invoke({
    "name": "dhcp-pool",
    "address_type": "ip_range",
    "address_value": "10.1.1.1-10.1.1.254",
    "folder": "Texas",
    "validated": False,
    "validation_result": ""
})

print(f"\nResult: {result3['validation_result']}")
print(f"Validated: {result3['validated']}")

print("\n✅ All three routing paths tested successfully!")
print("   The router correctly directed each type to its validator")

# Add graph structure analysis
print("\n📊 Graph Structure Analysis:")
print("=" * 60)
print("Nodes:")
print("  - router (decision point)")
print("  - validate_cidr (IP/netmask validator)")
print("  - validate_fqdn (FQDN validator)")
print("  - validate_range (IP range validator)")
print("\nRouting Logic:")
print("  address_type='ip_netmask' → validate_cidr")
print("  address_type='fqdn' → validate_fqdn")
print("  address_type='ip_range' → validate_range")
print("\n💡 This is 3-way routing - like a routing table with 3 next hops!")

---

## 7.5 Validation Success/Failure Routing Pattern

One of the most common routing patterns in network automation is handling **validation results**: routing validated objects to creation and invalid objects to error handling.

### The Scenario

When validating address objects before creation:
- **Valid objects** should proceed to creation
- **Invalid objects** should be logged as errors
- **Objects exceeding retry limit** should fail permanently

This pattern is critical for robust automation workflows!

### The ACL Analogy

Think of this like a security policy with inspection:

```
IF packet_validated == True THEN action: ALLOW → forward to destination
ELSE IF retry_count < max_retries THEN action: LOG → send to inspection again
ELSE action: DENY → drop packet and log error
```

Let's build this validation routing workflow!

In [ ]:
# Define state for validation routing
class ValidationRoutingState(TypedDict):
    """State for validation success/failure routing."""
    address_name: str
    address_value: str
    validated: bool
    error_message: str
    result: str

print("✅ ValidationRoutingState defined")
print("\nThis state tracks validation outcome and routes accordingly")

In [ ]:
# Node: Validate address format
def validate_address_format(state: ValidationRoutingState) -> dict:
    """Validate address format (CIDR notation)."""
    import re
    value = state["address_value"]
    
    # Validate CIDR format (e.g., 10.1.1.0/24)
    pattern = r'^(\d{1,3}\.){3}\d{1,3}/\d{1,2}$'
    is_valid = bool(re.match(pattern, value))
    
    error_msg = "" if is_valid else f"Invalid CIDR format: {value}"
    
    print(f"🔍 Validating: {value}")
    print(f"   Result: {'✅ Valid' if is_valid else '❌ Invalid'}")
    
    return {
        "validated": is_valid,
        "error_message": error_msg
    }


# Router function: Route based on validation result
def route_validation_result(state: ValidationRoutingState) -> Literal["success", "failed"]:
    """Route based on validation outcome.
    
    Returns:
        "success" if validated, "failed" if not validated
    """
    if state["validated"]:
        print("📍 Routing decision: Valid → success path (create object)")
        return "success"
    else:
        print("📍 Routing decision: Invalid → failed path (error handler)")
        return "failed"


# Node: Create address object (success path)
def create_address_object(state: ValidationRoutingState) -> dict:
    """Create address object after successful validation."""
    result = f"✅ Address '{state['address_name']}' created successfully with value {state['address_value']}"
    print(result)
    return {"result": result}


# Node: Handle validation error (failure path)
def handle_validation_error(state: ValidationRoutingState) -> dict:
    """Handle validation failure."""
    result = f"❌ Error: {state['error_message']}"
    print(result)
    return {"result": result}

print("✅ Validation routing nodes defined")
print("\nFlow: validate → router → [success: create | failed: error_handler]")

### Production Error Handling: Try-Catch Pattern

The validation function above shows basic error detection, but production workflows need **exception handling** to catch SCM API errors. Let's add a version with try-catch that demonstrates how to handle real SCM exceptions:

In [ ]:
# Node: Validate address format with try-catch error handling
def validate_address_with_exception_handling(state: ValidationRoutingState) -> dict:
    """Validate address format with comprehensive exception handling.
    
    This demonstrates production error handling patterns for SCM automation.
    """
    import re
    
    try:
        value = state["address_value"]
        
        # Validate CIDR format (e.g., 10.1.1.0/24)
        pattern = r'^(\d{1,3}\.){3}\d{1,3}/\d{1,2}$'
        is_valid = bool(re.match(pattern, value))
        
        if not is_valid:
            # Simulate SCM InvalidObjectError for bad format
            raise ValueError(f"InvalidObjectError: Invalid CIDR format '{value}'")
        
        # Additional validation: Check octet ranges (0-255)
        ip_part = value.split('/')[0]
        octets = ip_part.split('.')
        
        for octet in octets:
            if not (0 <= int(octet) <= 255):
                raise ValueError(f"InvalidObjectError: Octet value '{octet}' out of range (0-255)")
        
        # Check subnet mask range (0-32 for IPv4)
        mask = int(value.split('/')[1])
        if not (0 <= mask <= 32):
            raise ValueError(f"InvalidObjectError: Subnet mask '{mask}' out of range (0-32)")
        
        print(f"🔍 Validating: {value}")
        print(f"   Result: ✅ Valid")
        
        return {
            "validated": True,
            "error_message": ""
        }
    
    except ValueError as e:
        # Catch validation errors (simulating SCM InvalidObjectError)
        error_msg = str(e)
        print(f"🔍 Validating: {value}")
        print(f"   Result: ❌ Invalid - {error_msg}")
        
        return {
            "validated": False,
            "error_message": error_msg
        }
    
    except IndexError as e:
        # Catch parsing errors (malformed input)
        error_msg = f"InvalidObjectError: Malformed CIDR notation '{value}'"
        print(f"🔍 Validating: {value}")
        print(f"   Result: ❌ Invalid - {error_msg}")
        
        return {
            "validated": False,
            "error_message": error_msg
        }
    
    except Exception as e:
        # Catch unexpected errors
        error_msg = f"Unexpected error during validation: {str(e)}"
        print(f"🔍 Validating: {value}")
        print(f"   Result: ❌ Error - {error_msg}")
        
        return {
            "validated": False,
            "error_message": error_msg
        }

print("✅ validate_address_with_exception_handling defined!")
print("\n💡 Production error handling pattern:")
print("   1. Wrap validation logic in try-catch block")
print("   2. Catch specific exceptions (ValueError for validation errors)")
print("   3. Catch parsing errors (IndexError for malformed input)")
print("   4. Catch unexpected errors (Exception as fallback)")
print("   5. Return validated=False and error_message on any exception")
print("   6. In production, this would catch actual SCM exceptions:")
print("      - InvalidObjectError")
print("      - NameNotUniqueError")
print("      - ObjectNotPresentError")

In [ ]:
# Build validation routing graph
validation_routing_graph = StateGraph(ValidationRoutingState)

# Add validation node
validation_routing_graph.add_node("validate", validate_address_format)

# Add router node with lambda passthrough
validation_routing_graph.add_node("router", lambda state: state)

# Add processing nodes
validation_routing_graph.add_node("create_object", create_address_object)
validation_routing_graph.add_node("error_handler", handle_validation_error)

# Connect START → validate
validation_routing_graph.set_entry_point("validate")

# Connect validate → router
validation_routing_graph.add_edge("validate", "router")

# Add conditional edges: router → [success or failed]
validation_routing_graph.add_conditional_edges(
    source="router",
    path=route_validation_result,
    path_map={
        "success": "create_object",
        "failed": "error_handler"
    }
)

# Connect both paths to END
validation_routing_graph.add_edge("create_object", END)
validation_routing_graph.add_edge("error_handler", END)

# Compile
validation_routing_app = validation_routing_graph.compile()

print("✅ Validation routing graph compiled")
print("\n💡 2-way routing: success → create_object, failed → error_handler")

In [ ]:
# Visualize validation routing graph
display(Image(validation_routing_app.get_graph().draw_mermaid_png()))

print("\n📊 Graph Structure Analysis:")
print("=" * 60)
print("Nodes:")
print("  - validate (validation logic)")
print("  - router (decision point)")
print("  - create_object (success path)")
print("  - error_handler (failure path)")
print("\nFlow:")
print("  START → validate → router → [create_object OR error_handler] → END")
print("\nDecision Logic:")
print("  validated=True → success path (create object)")
print("  validated=False → failed path (error handler)")
print("\n💡 This is classic success/failure routing!")

In [ ]:
# Test validation success path
print("=" * 60)
print("TEST 1: Valid Address (Success Path)")
print("=" * 60)

result_valid = validation_routing_app.invoke({
    "address_name": "internal-network",
    "address_value": "10.1.1.0/24",
    "validated": False,
    "error_message": "",
    "result": ""
})

print(f"\nFinal result: {result_valid['result']}")
print(f"Validated: {result_valid['validated']}")

print("\n" + "=" * 60)
print("TEST 2: Invalid Address (Failure Path)")
print("=" * 60)

result_invalid = validation_routing_app.invoke({
    "address_name": "bad-network",
    "address_value": "not-a-valid-cidr",
    "validated": False,
    "error_message": "",
    "result": ""
})

print(f"\nFinal result: {result_invalid['result']}")
print(f"Validated: {result_invalid['validated']}")

print("\n✅ Both validation paths tested successfully!")
print("   Valid addresses → create_object")
print("   Invalid addresses → error_handler")

---

## 7.6 Error Handling vs Normal Processing Paths

Another critical routing pattern is separating **error handling** from **normal processing** based on API responses or operation outcomes.

### The Scenario

When making API calls to SCM:
- **Successful responses (200)** should proceed to normal processing
- **Failed responses (4xx, 5xx)** should route to error handling
- Error handlers can log issues, send notifications, or trigger retries

### Network Analogy: Link Status Detection

Think of this like a network interface monitoring its link status:

```
IF link_status == UP THEN route: forward traffic to destination
ELSE IF link_status == DOWN THEN route: switch to backup path / log error
```

Or like ICMP reachability checks in routing protocols:

```
IF next_hop_reachable == True THEN action: use primary path
ELSE action: use backup path and trigger alert
```

Let's build an error handling router!

In [ ]:
# Define state for error handling routing
class APICallState(TypedDict):
    """State for API error handling routing."""
    address_name: str
    address_value: str
    api_status_code: int
    api_error: str
    result: str

print("✅ APICallState defined")
print("\nThis state tracks API responses and routes to success or error handlers")

In [ ]:
# Node: Simulate API call
def call_scm_api(state: APICallState) -> dict:
    """Simulate SCM API call to create address object."""
    import random
    
    # Simulate API call - randomly succeed or fail for demo
    status_code = random.choice([200, 200, 200, 400, 500])  # 60% success rate
    
    if status_code == 200:
        error = ""
        print(f"✅ API Call: POST /address-objects → Status {status_code}")
    elif status_code == 400:
        error = "Bad Request: Invalid address format"
        print(f"❌ API Call: POST /address-objects → Status {status_code}")
    else:
        error = "Internal Server Error: Service unavailable"
        print(f"❌ API Call: POST /address-objects → Status {status_code}")
    
    return {
        "api_status_code": status_code,
        "api_error": error
    }


# Router function: Route based on API response
def route_api_response(state: APICallState) -> Literal["success", "error"]:
    """Route based on API response status code.
    
    Returns:
        "success" for 2xx responses, "error" for 4xx/5xx
    """
    if state["api_status_code"] == 200:
        print("📍 Routing decision: HTTP 200 → success path")
        return "success"
    else:
        print(f"📍 Routing decision: HTTP {state['api_status_code']} → error path")
        return "error"


# Node: Process successful response
def process_success(state: APICallState) -> dict:
    """Process successful API response."""
    result = f"✅ SUCCESS: Address '{state['address_name']}' ({state['address_value']}) created successfully"
    print(result)
    return {"result": result}


# Node: Handle API error
def handle_api_error(state: APICallState) -> dict:
    """Handle API error response."""
    result = f"❌ ERROR: Failed to create address '{state['address_name']}' - {state['api_error']}"
    print(result)
    return {"result": result}

print("✅ API error handling nodes defined")
print("\nFlow: api_call → router → [success: process | error: error_handler]")

In [ ]:
# Build API error handling graph
api_error_graph = StateGraph(APICallState)

# Add API call node
api_error_graph.add_node("api_call", call_scm_api)

# Add router node with lambda passthrough
api_error_graph.add_node("router", lambda state: state)

# Add processing nodes
api_error_graph.add_node("process_success", process_success)
api_error_graph.add_node("error_handler", handle_api_error)

# Connect START → api_call
api_error_graph.set_entry_point("api_call")

# Connect api_call → router
api_error_graph.add_edge("api_call", "router")

# Add conditional edges: router → [success or error]
api_error_graph.add_conditional_edges(
    source="router",
    path=route_api_response,
    path_map={
        "success": "process_success",
        "error": "error_handler"
    }
)

# Connect both paths to END
api_error_graph.add_edge("process_success", END)
api_error_graph.add_edge("error_handler", END)

# Compile
api_error_app = api_error_graph.compile()

print("✅ API error handling graph compiled")
print("\n💡 Routes based on HTTP status: 200 → success, 4xx/5xx → error")

In [ ]:
# Visualize API error handling graph
display(Image(api_error_app.get_graph().draw_mermaid_png()))

print("\n📊 Graph Structure Analysis:")
print("=" * 60)
print("Nodes:")
print("  - api_call (makes SCM API request)")
print("  - router (decision point)")
print("  - process_success (normal processing)")
print("  - error_handler (error handling)")
print("\nFlow:")
print("  START → api_call → router → [process_success OR error_handler] → END")
print("\nDecision Logic:")
print("  status_code=200 → success path (normal processing)")
print("  status_code=4xx/5xx → error path (error handler)")
print("\n💡 Separates happy path from error handling!")

In [ ]:
# Test both success and error paths
print("=" * 60)
print("TEST 1: Successful API Call")
print("=" * 60)

result_success = api_error_app.invoke({
    "address_name": "web-server",
    "address_value": "10.1.1.10/32",
    "api_status_code": 0,  # Will be set by api_call node
    "api_error": "",
    "result": ""
})

print(f"\nFinal result: {result_success['result']}")
print(f"Status code: {result_success['api_status_code']}")

print("\n" + "=" * 60)
print("TEST 2: Failed API Call (simulated by re-running)")
print("=" * 60)
print("Note: The API call randomly succeeds or fails for demonstration.")
print("Run this cell multiple times to see both paths!\n")

# Run multiple times to demonstrate both paths
for i in range(3):
    print(f"\n--- Attempt {i+1} ---")
    result = api_error_app.invoke({
        "address_name": f"test-server-{i+1}",
        "address_value": f"10.1.1.{i+10}/32",
        "api_status_code": 0,
        "api_error": "",
        "result": ""
    })
    print(f"Status: {result['api_status_code']}")

print("\n✅ API error handling routing demonstrated!")
print("   Success (200) → process_success")
print("   Error (4xx/5xx) → error_handler")

---

## 7.7 SCM Operation Routing (Create/Update/Delete)

A common pattern in SCM automation is routing different CRUD operations to appropriate handlers. Different operations require different API endpoints and validation logic.

### The Scenario

When automating SCM workflows, you need to handle:
- **CREATE**: POST new objects to SCM
- **UPDATE**: PUT/PATCH existing objects
- **DELETE**: DELETE objects from SCM
- **READ**: GET object details (optional)

Each operation routes to a different handler with appropriate logic.

### Network Analogy: NETCONF Operation Routing

Think of this like NETCONF operations on network devices:

```
IF operation == "edit-config" THEN route_to: config_manager
ELSE IF operation == "get-config" THEN route_to: config_reader
ELSE IF operation == "delete-config" THEN route_to: config_deleter
```

Or like REST API method routing:

```
POST /api/address-objects → create_handler
PUT /api/address-objects/{id} → update_handler  
DELETE /api/address-objects/{id} → delete_handler
```

Let's build an SCM operation router!

In [ ]:
# Define state for SCM operation routing
class SCMOperationState(TypedDict):
    """State for SCM CRUD operation routing."""
    operation_type: str  # "create", "update", or "delete"
    address_name: str
    address_value: str
    address_id: str  # For update/delete operations
    folder: str
    result: str

print("✅ SCMOperationState defined")
print("\nThis state tracks operation type and routes to appropriate handler")

In [ ]:
# Router function: Route by operation type
def route_by_operation(state: SCMOperationState) -> Literal["create", "update", "delete"]:
    """Route based on SCM operation type.
    
    Returns:
        "create", "update", or "delete" based on operation_type
    """
    operation = state["operation_type"]
    
    if operation == "create":
        print(f"📍 Routing decision: {operation} → create_handler")
        return "create"
    elif operation == "update":
        print(f"📍 Routing decision: {operation} → update_handler")
        return "update"
    else:  # delete
        print(f"📍 Routing decision: {operation} → delete_handler")
        return "delete"


# Node: Handle CREATE operation
def handle_create_operation(state: SCMOperationState) -> dict:
    """Handle address object creation."""
    result = (
        f"✅ CREATE: Address object '{state['address_name']}' "
        f"created in '{state['folder']}' folder\\n"
        f"   Value: {state['address_value']}\\n"
        f"   API: POST /config/objects/v1/addresses"
    )
    print(result)
    return {"result": result}


# Node: Handle UPDATE operation
def handle_update_operation(state: SCMOperationState) -> dict:
    """Handle address object update."""
    result = (
        f"✅ UPDATE: Address object '{state['address_name']}' (ID: {state['address_id']}) updated\\n"
        f"   New value: {state['address_value']}\\n"
        f"   API: PUT /config/objects/v1/addresses/{state['address_id']}"
    )
    print(result)
    return {"result": result}


# Node: Handle DELETE operation
def handle_delete_operation(state: SCMOperationState) -> dict:
    """Handle address object deletion."""
    result = (
        f"✅ DELETE: Address object '{state['address_name']}' (ID: {state['address_id']}) deleted\\n"
        f"   API: DELETE /config/objects/v1/addresses/{state['address_id']}"
    )
    print(result)
    return {"result": result}

print("✅ SCM operation handlers defined")
print("\nOperations: CREATE (POST), UPDATE (PUT), DELETE (DELETE)")

In [ ]:
# Build SCM operation routing graph
scm_operation_graph = StateGraph(SCMOperationState)

# Add router node with lambda passthrough
scm_operation_graph.add_node("router", lambda state: state)

# Add operation handler nodes
scm_operation_graph.add_node("create_handler", handle_create_operation)
scm_operation_graph.add_node("update_handler", handle_update_operation)
scm_operation_graph.add_node("delete_handler", handle_delete_operation)

# Set entry point
scm_operation_graph.set_entry_point("router")

# Add conditional edges with 3-way routing
scm_operation_graph.add_conditional_edges(
    source="router",
    path=route_by_operation,
    path_map={
        "create": "create_handler",
        "update": "update_handler",
        "delete": "delete_handler"
    }
)

# Connect all handlers to END
scm_operation_graph.add_edge("create_handler", END)
scm_operation_graph.add_edge("update_handler", END)
scm_operation_graph.add_edge("delete_handler", END)

# Compile
scm_operation_app = scm_operation_graph.compile()

print("✅ SCM operation routing graph compiled")
print("\n💡 3-way CRUD routing: create/update/delete")

In [ ]:
# Visualize SCM operation routing graph
display(Image(scm_operation_app.get_graph().draw_mermaid_png()))

print("\n📊 Graph Structure Analysis:")
print("=" * 60)
print("Nodes:")
print("  - router (operation decision point)")
print("  - create_handler (POST new objects)")
print("  - update_handler (PUT existing objects)")
print("  - delete_handler (DELETE objects)")
print("\nFlow:")
print("  START → router → [create/update/delete handler] → END")
print("\nRouting Logic:")
print("  operation_type='create' → create_handler (POST)")
print("  operation_type='update' → update_handler (PUT)")
print("  operation_type='delete' → delete_handler (DELETE)")
print("\n💡 Classic CRUD routing pattern for SCM automation!")

In [ ]:
# Test all three SCM operation paths
print("=" * 60)
print("TEST 1: CREATE Operation")
print("=" * 60)

result_create = scm_operation_app.invoke({
    "operation_type": "create",
    "address_name": "web-server",
    "address_value": "10.1.1.10/32",
    "address_id": "",
    "folder": "Texas",
    "result": ""
})

print(f"\nOperation result:\\n{result_create['result']}")

print("\n" + "=" * 60)
print("TEST 2: UPDATE Operation")
print("=" * 60)

result_update = scm_operation_app.invoke({
    "operation_type": "update",
    "address_name": "web-server",
    "address_value": "10.1.1.20/32",  # Changed IP
    "address_id": "abc-123-def-456",
    "folder": "Texas",
    "result": ""
})

print(f"\nOperation result:\\n{result_update['result']}")

print("\n" + "=" * 60)
print("TEST 3: DELETE Operation")
print("=" * 60)

result_delete = scm_operation_app.invoke({
    "operation_type": "delete",
    "address_name": "old-server",
    "address_value": "",
    "address_id": "xyz-789-uvw-012",
    "folder": "Texas",
    "result": ""
})

print(f"\nOperation result:\\n{result_delete['result']}")

print("\n✅ All three CRUD operation paths tested successfully!")
print("   CREATE → create_handler (POST)")
print("   UPDATE → update_handler (PUT)")
print("   DELETE → delete_handler (DELETE)")

---

## 8. Exercise: Build Your Own Router

Now it's your turn! Apply what you've learned to build a comprehensive conditional routing workflow.

### Challenge: Multi-Path Configuration Router

Build a graph that demonstrates the key routing patterns you've learned in this notebook.

**Part A: Configuration Change Type Router (3-way routing)**

Create a router that handles different SCM configuration types:

1. **Define State** with these fields:
   - `change_type`: str ("security_policy", "nat_rule", or "address_object")
   - `change_description`: str
   - `folder`: str
   - `processed`: bool
   - `handler_used`: str

2. **Create Router Function** that:
   - Returns `Literal["policy_path", "nat_path", "address_path"]`
   - Routes based on `change_type` field

3. **Create Three Handler Nodes**:
   - `handle_security_policy`: Sets handler_used to "security policy handler"
   - `handle_nat_rule`: Sets handler_used to "NAT rule handler"
   - `handle_address_object`: Sets handler_used to "address object handler"

4. **Build Graph** with:
   - Router node using lambda passthrough
   - Conditional edges with 3-way routing
   - All handlers connecting to END

5. **Test** all three paths with different change_type values

**Expected Output**:

```python
# Input
{"change_type": "security_policy", "change_description": "Allow web traffic", ...}

# Output
{..., "handler_used": "security policy handler", "processed": True}
```

**Part B (Bonus): Add Validation and Error Handling**

If you want an extra challenge, extend your solution to include:

1. **Validation Router**: Before processing, validate the configuration
   - Valid configurations → route to appropriate handler
   - Invalid configurations → route to error handler

2. **Error Handling**: Add error detection after processing
   - Success (simulated) → route to completion
   - Failure (simulated) → route to retry or error logging

**Key Points**:
- Use `lambda state: state` for router nodes
- Router functions return edge name (string)
- Use `add_conditional_edges()` with path_map
- Use Literal return types for type safety
- Test all routing paths!

**Hints**:
- Start with Part A to practice the basic pattern
- For Part B, you'll need multiple routers in sequence
- Think about the flow: validate → route by type → handle → check result → route to success/error

Try it below!

In [ ]:
# Your code here!
# Build the configuration change routing workflow

# Step 1: Define ConfigChangeState


# Step 2: Create route_by_change_type router function


# Step 3: Create handle_security_policy node function


# Step 4: Create handle_nat_rule node function


# Step 5: Create handle_address_object node function


# Step 6: Build graph


# Step 7: Add router node with lambda passthrough


# Step 8: Add handler nodes


# Step 9: Set entry point


# Step 10: Add conditional edge with path_map


# Step 11: Add edges from handlers to END


# Step 12: Compile


# Step 13: Visualize


# Step 14: Test with security_policy


# Step 15: Test with nat_rule


# Step 16: Test with address_object


### 8.1 Exercise Solution

Below is the complete solution for the configuration change routing workflow. Try the exercise yourself first before looking at this solution!

In [ ]:
# SOLUTION: Configuration Change Type Router

# Step 1: Define ConfigChangeState
class ConfigChangeState(TypedDict):
    """State for configuration change type routing."""
    change_type: str           # "security_policy", "nat_rule", or "address_object"
    change_description: str    # Description of the change
    folder: str                # Target SCM folder
    processed: bool            # Has change been processed?
    handler_used: str          # Which handler processed this change

print("✅ Step 1: ConfigChangeState defined with 5 fields")

# Step 2: Create router function
def route_by_change_type(state: ConfigChangeState) -> Literal["policy_path", "nat_path", "address_path"]:
    """Route based on configuration change type.
    
    Args:
        state: Current workflow state
        
    Returns:
        Literal edge name indicating which handler to use
    """
    change_type = state["change_type"]
    
    if change_type == "security_policy":
        print(f"📍 Routing decision: {change_type} → policy_path")
        return "policy_path"
    elif change_type == "nat_rule":
        print(f"📍 Routing decision: {change_type} → nat_path")
        return "nat_path"
    else:  # address_object
        print(f"📍 Routing decision: {change_type} → address_path")
        return "address_path"

print("✅ Step 2: route_by_change_type router function created")

# Step 3: Create handle_security_policy node
def handle_security_policy(state: ConfigChangeState) -> dict:
    """Handle security policy changes.
    
    Args:
        state: Current workflow state
        
    Returns:
        dict: Partial state update marking change as processed
    """
    desc = state["change_description"]
    folder = state["folder"]
    
    print(f"🔐 Processing security policy change in '{folder}' folder")
    print(f"   Description: {desc}")
    
    return {
        "handler_used": "security policy handler",
        "processed": True
    }

print("✅ Step 3: handle_security_policy node created")

# Step 4: Create handle_nat_rule node
def handle_nat_rule(state: ConfigChangeState) -> dict:
    """Handle NAT rule changes.
    
    Args:
        state: Current workflow state
        
    Returns:
        dict: Partial state update marking change as processed
    """
    desc = state["change_description"]
    folder = state["folder"]
    
    print(f"🔀 Processing NAT rule change in '{folder}' folder")
    print(f"   Description: {desc}")
    
    return {
        "handler_used": "NAT rule handler",
        "processed": True
    }

print("✅ Step 4: handle_nat_rule node created")

# Step 5: Create handle_address_object node
def handle_address_object(state: ConfigChangeState) -> dict:
    """Handle address object changes.
    
    Args:
        state: Current workflow state
        
    Returns:
        dict: Partial state update marking change as processed
    """
    desc = state["change_description"]
    folder = state["folder"]
    
    print(f"📍 Processing address object change in '{folder}' folder")
    print(f"   Description: {desc}")
    
    return {
        "handler_used": "address object handler",
        "processed": True
    }

print("✅ Step 5: handle_address_object node created")

# Step 6: Build graph
config_change_graph = StateGraph(ConfigChangeState)

print("✅ Step 6: Graph created")

# Step 7: Add router node with lambda passthrough
config_change_graph.add_node("router", lambda state: state)

print("✅ Step 7: Router node added with lambda passthrough")

# Step 8: Add handler nodes
config_change_graph.add_node("policy_handler", handle_security_policy)
config_change_graph.add_node("nat_handler", handle_nat_rule)
config_change_graph.add_node("address_handler", handle_address_object)

print("✅ Step 8: All handler nodes added")

# Step 9: Set entry point
config_change_graph.set_entry_point("router")

print("✅ Step 9: Entry point set to router")

# Step 10: Add conditional edge with path_map
config_change_graph.add_conditional_edges(
    source="router",
    path=route_by_change_type,
    path_map={
        "policy_path": "policy_handler",
        "nat_path": "nat_handler",
        "address_path": "address_handler"
    }
)

print("✅ Step 10: Conditional edges added with 3-way routing")

# Step 11: Add edges from handlers to END
config_change_graph.add_edge("policy_handler", END)
config_change_graph.add_edge("nat_handler", END)
config_change_graph.add_edge("address_handler", END)

print("✅ Step 11: All handlers connected to END")

# Step 12: Compile
config_change_app = config_change_graph.compile()

print("✅ Step 12: Graph compiled successfully")

# Step 13: Visualize
print("\n💡 Step 13: Visualizing the configuration change router...")
display(Image(config_change_app.get_graph().draw_mermaid_png()))

print("\n📊 Graph Structure Analysis:")
print("=" * 60)
print("Nodes:")
print("  - router (decision point)")
print("  - policy_handler (security policy changes)")
print("  - nat_handler (NAT rule changes)")
print("  - address_handler (address object changes)")
print("\nRouting Logic:")
print("  change_type='security_policy' → policy_handler")
print("  change_type='nat_rule' → nat_handler")
print("  change_type='address_object' → address_handler")
print("\n💡 3-way routing for different SCM object types!")

# Step 14: Test with security_policy
print("\n" + "=" * 60)
print("TEST 1: Security Policy Change")
print("=" * 60)

result1 = config_change_app.invoke({
    "change_type": "security_policy",
    "change_description": "Allow web traffic from internal network",
    "folder": "Texas",
    "processed": False,
    "handler_used": ""
})

print(f"\n✅ Result:")
print(f"   Handler used: {result1['handler_used']}")
print(f"   Processed: {result1['processed']}")

# Step 15: Test with nat_rule
print("\n" + "=" * 60)
print("TEST 2: NAT Rule Change")
print("=" * 60)

result2 = config_change_app.invoke({
    "change_type": "nat_rule",
    "change_description": "Source NAT for internal network to internet",
    "folder": "Texas",
    "processed": False,
    "handler_used": ""
})

print(f"\n✅ Result:")
print(f"   Handler used: {result2['handler_used']}")
print(f"   Processed: {result2['processed']}")

# Step 16: Test with address_object
print("\n" + "=" * 60)
print("TEST 3: Address Object Change")
print("=" * 60)

result3 = config_change_app.invoke({
    "change_type": "address_object",
    "change_description": "Add new web server address 10.1.1.10/32",
    "folder": "Texas",
    "processed": False,
    "handler_used": ""
})

print(f"\n✅ Result:")
print(f"   Handler used: {result3['handler_used']}")
print(f"   Processed: {result3['processed']}")

print("\n" + "=" * 60)
print("✅ All 16 steps completed successfully!")
print("   All three routing paths tested and working correctly")
print("=" * 60)

---

## 9. What's Next

Congratulations! You've mastered conditional routing in LangGraph. You can now build workflows that make intelligent decisions based on state conditions.

### What You've Learned

In this notebook, you learned to:
- ✅ Use `add_conditional_edges()` to create branching workflows
- ✅ Write router functions with `Literal` return types
- ✅ Apply the lambda passthrough pattern (`lambda state: state`)
- ✅ Route based on state conditions (folder type, address type)
- ✅ Think about routing like network ACLs and routing tables

### Coming in Notebook 107: Looping Workflows

You've learned to build:
- Linear workflows (Notebook 104): A → B → C
- Branching workflows (Notebook 106): A → (B or C) → D

But what about workflows that need to **repeat** steps? That's where **looping** comes in!

In Notebook 107, you'll learn:
- How to create loops in LangGraph (going back to previous nodes)
- Conditional routing that can loop back (retry logic)
- Building workflows with termination conditions
- Real-world patterns like:
  - **Retry loops**: Try up to 3 times, then fail
  - **Polling loops**: Check HA sync status until synced
  - **Batch processing**: Process list of objects one at a time
  - **Approval loops**: Keep requesting approval until approved/rejected

### Preview: Looping Pattern

Here's a taste of what you'll build:

```python
def should_retry(state) -> Literal["retry", "success", "failed"]:
    if state["validated"]:
        return "success"  # → END
    elif state["retry_count"] < 3:
        return "retry"    # → Loop back to validate
    else:
        return "failed"   # → error_handler

# This creates a loop!
graph.add_conditional_edges(
    source="check_result",
    path=should_retry,
    path_map={
        "retry": "validate",      # ← Loops back!
        "success": END,
        "failed": "error_handler"
    }
)
```

Exciting stuff ahead! 🚀

---

## 10. Summary

### Key Concepts Covered

1. **Conditional Routing**
   - Enables workflows to make decisions based on state
   - Creates branching paths instead of linear sequences
   - Like ACL rules or routing table lookups

2. **The Three Components**
   - **Router Function**: Examines state, returns edge name
   - **Lambda Passthrough**: `lambda state: state` for router node
   - **Conditional Edge**: `add_conditional_edges()` connects them

3. **Router Functions**
   - Return string (edge name), not dict (state update)
   - Use `Literal` type hints for type safety
   - Contain decision logic (if/elif/else)
   - Don't modify state, only decide routing

4. **Lambda Passthrough Pattern**
   - `graph.add_node("router", lambda state: state)`
   - Satisfies LangGraph requirement (nodes return state)
   - Allows router function to make decisions separately
   - Standard pattern for all conditional routing

5. **add_conditional_edges() API**
   ```python
   graph.add_conditional_edges(
       source="router",        # Node making decision
       path=router_function,   # Function with logic
       path_map={              # Edge name → destination
           "edge_a": "node_a",
           "edge_b": "node_b"
       }
   )
   ```

6. **Network ACL Analogy**
   - State = Packet flowing through network
   - Router function = ACL rule matching logic
   - Edge name = Action (permit → node_a, deny → node_b)
   - path_map = Routing table (destination → next hop)

### Real-World Applications

You can now build:
- Environment-based routing (dev vs staging vs prod)
- Type-based validation (IP vs FQDN vs Range)
- Success/failure paths (error handling)
- Approval workflows (requires approval vs direct)
- HA vs standalone logic (different upgrade paths)

### Next Steps

- **Notebook 107**: Looping workflows and retry logic
- **Future Notebooks**: LLM-powered routers that make AI-based decisions

### Remember

The complete pattern:

```python
# 1. Define router function with Literal return type
def router(state: MyState) -> Literal["path_a", "path_b"]:
    if state["condition"]:
        return "path_a"
    return "path_b"

# 2. Add router node with lambda passthrough
graph.add_node("router", lambda state: state)

# 3. Add conditional edges with path_map
graph.add_conditional_edges(
    source="router",
    path=router,
    path_map={"path_a": "node_a", "path_b": "node_b"}
)
```

You're now ready to build sophisticated, decision-making workflows! 🎉